In [2]:
import argparse
import copy
import logging
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import time
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from preactresnet import PreActResNet18
from wideresnet import WideResNet,WideResNet1,WideResNet_save
from utils_plus import (upper_limit, lower_limit, std, clamp, get_loaders,
    attack_pgd, evaluate_pgd, evaluate_standard, normalize)
from autoattack import AutoAttack
import torch.utils.data as data
device = torch.device('cuda:0') 
from torchdiffeq import odeint_adjoint as odeint

endtime = 5

fc_dim = 64
act = torch.sin 
f_coeffi = -1
layernum = 0
tol = 1e-3


saved_temp = torch.load('./EXP/nips_model/full.pth')
statedic_temp = saved_temp['state_dict']




def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch-size', default=128, type=int)
    parser.add_argument('--data-dir', default='../cifar-data', type=str)
    parser.add_argument('--epsilon', default=8, type=int)
    parser.add_argument('--out-dir', default='train_fgsm_output', type=str, help='Output directory')
    parser.add_argument('--seed', default=0, type=int, help='Random seed')
    return parser.parse_args()


def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)


def accuracy(model, dataset_loader):
    total_correct = 0
    for x, y in dataset_loader:
        x = x.to(device)
        y = one_hot(np.array(y.numpy()), 10)

        target_class = np.argmax(y, axis=1)
        predicted_class = np.argmax(model(x)[0].cpu().detach().numpy(), axis=1)
        total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(dataset_loader.dataset)



def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)





class ConcatFC(nn.Module):

    def __init__(self, dim_in, dim_out):
        super(ConcatFC, self).__init__()
        self._layer = nn.Linear(dim_in, dim_out)
    def forward(self, t, x):
        return self._layer(x)


    
class ODEfunc_mlp(nn.Module):

    def __init__(self, dim):
        super(ODEfunc_mlp, self).__init__()
        self.fc1 = ConcatFC(64, 256)
        self.act1 = act
        self.fc2 = ConcatFC(256, 256)
        self.act2 = act
        self.fc3 = ConcatFC(256, 64)
        self.act3 = act
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = f_coeffi*self.fc1(t, x)
        out = self.act1(out)
        out = f_coeffi*self.fc2(t, out)
        out = self.act2(out)
        out = f_coeffi*self.fc3(t, out)
        out = self.act3(out)
        
        return out

    
    
class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, endtime]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=tol, atol=tol)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value


class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)

    
class MLP_OUT_Linear(nn.Module):

    def __init__(self):
        super(MLP_OUT_Linear, self).__init__()
        self.fc0 = nn.Linear(fc_dim, 10)
    def forward(self, input_):
#         h1 = F.relu(self.fc0(input_))
        h1 = self.fc0(input_)
        return h1


def inf_generator(iterable):
    """Allows training with DataLoaders in a single infinite loop:
        for i, (x, y) in enumerate(inf_generator(train_loader)):
    """
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch-size', default=128, type=int)
    parser.add_argument('--data-dir', default='../cifar-data', type=str)
    parser.add_argument('--epsilon', default=8, type=int)
    parser.add_argument('--out-dir', default='train_fgsm_output', type=str, help='Output directory')
    parser.add_argument('--seed', default=0, type=int, help='Random seed')
#     return parser.parse_args()
    return parser.parse_args(args=[])


def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)


def accuracy(model, dataset_loader):
    total_correct = 0
    for x, y in dataset_loader:
        x = x.to(device)
        y = one_hot(np.array(y.numpy()), 10)

        target_class = np.argmax(y, axis=1)
        predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)
        total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(dataset_loader.dataset)



def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)


        

args = get_args()
nepochs = 100
batches_per_epoch = 128

np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)




model = WideResNet_save(fc_dim, 34, 10, widen_factor=10, dropRate=0.0)
odefunc = ODEfunc_mlp(0)
feature_layers = [ODEBlock(odefunc)] 
fc_layers = [MLP_OUT_Linear()]


model = nn.Sequential(model, *feature_layers, *fc_layers).to(device)


model.load_state_dict(statedic_temp)

print(model)
model.to(device)
model.eval()



train_loader, test_loader, train_loader__, test_dataset = get_loaders(args.data_dir, args.batch_size)
class mnist_samples(data.Dataset):
    def __init__(self, dataset, leng, iid):
        self.dataset = dataset
        self.len = leng
        self.iid = iid
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        x,y = self.dataset[idx+self.len*self.iid]
        return x,y







#################### load original pretrained model and generate aa attack adv samples ############            return self.len

    # x_test, y_test = load_cifar100(n_examples=100)
samplenum = 512
test_samples = mnist_samples(test_dataset,samplenum,1)
test_loader_samples = data.DataLoader(test_samples, batch_size=100, shuffle=False, num_workers=2, drop_last=False)
#     x_test, y_test = _load_dataset(test_samples,300)




print('run_standard_evaluation_individual', 'Linf')
# print('run_standard_evaluation_individual', 'L2')


#     test_loader = test_loader
test_loader = test_loader_samples

l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)



best_state_dict = torch.load('./EXP/pre_train_model/wide10_trades_eps8_tricks.pt')
# Evaluation
model_test = WideResNet(34, 10, widen_factor=10, dropRate=0.0).cuda()
if 'state_dict' in best_state_dict.keys():
    model_test.load_state_dict(best_state_dict['state_dict'])
else:
    model_test.load_state_dict(best_state_dict)
model_test.float()
model_test.eval()



epsilon = 8 / 255.
adversary = AutoAttack(model_test, norm='Linf', eps=epsilon, version='standard')

#     epsilon = 0.5
#     adversary = AutoAttack(new_model, norm='L2', eps=epsilon, version='standard')


# adversary.attacks_to_run = ['apgd-ce', 'apgd-t']

X_adv = adversary.run_standard_evaluation(x_test, y_test, bs=512)
######################################################################################################



total_correct = 0
x = X_adv.to(device)

target_class = y_test
predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)


# print(target_class)
# print("==========")
# print(predicted_class)

print("transfer attack acc using adv examples generated from original pretrained model without sodef: ", sum(target_class.numpy()== predicted_class)/samplenum)

    



R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
Use activation of ReLU
Sequential(
  (0): WideResNet_save(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (block1): NetworkBlock(
      (layer): Sequential(
        (0): BasicBlock(
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (relu1): ReLU(inplace=True)
          (relu2): ReLU(inplace=True)
          (convShortcut): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BasicBlock(
          (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv1): Conv2d(160, 160, kerne